In [10]:
import ppigrf
from datetime import datetime
import numpy as np
import pandas as pd
from emp.geometry import Point
from emp.constants import EARTH_RADIUS
from emp.region_scan import (
    contour_plot,
    data_dic_to_xyz,
    region_scan,
    folium_plot,
)
import warnings
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler
import pickle

# plot settings
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 5.0
plt.rcParams['xtick.minor.size'] = 3.0
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['lines.linewidth'] = 2
plt.rc('font', family='serif',size=16)
matplotlib.rc('text', usetex=True)
matplotlib.rc('legend', fontsize=16)
matplotlib.rcParams['axes.prop_cycle'] = cycler(
    color=['#E24A33', '#348ABD', '#988ED5', '#777777', '#FBC15E', '#8EBA42', '#FFB5B8']
    )
matplotlib.rcParams.update(
    {"axes.grid":True,
    "grid.alpha":0.75,
    "grid.linewidth":0.5}
    )
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
import importlib
import emp.region_scan

importlib.reload(emp.region_scan)
from emp.region_scan import folium_plot

In [3]:
warnings.filterwarnings("ignore", category=FutureWarning, message=".*'unit' keyword in TimedeltaIndex construction.*")

In [4]:
df = pd.read_csv("Nuke tests - Sheet1.csv")
df

,Name,Country,Date,Yield (kt),Latitude/Longitude,Latitude,Longitude,Altitude (km),Reference
0,127 K2 (Joe 109),USSR,10/27/1961,1.2,46.408°N 72.237°E,46.408000,72.237000,180,https://en.wikipedia.org/wiki/Soviet_Project_K...
1,128 K1 (Joe 105),USSR,10/27/1961,1.2,46.7°N 69.6°E,46.700000,69.600000,300,https://en.wikipedia.org/wiki/Soviet_Project_K...
2,184 K3 (Joe 157),USSR,10/22/1962,300.0,47.76469°N 63.95136°E,47.764690,63.951360,290,https://en.wikipedia.org/wiki/Soviet_Project_K...
3,187 K4 (Joe 160),USSR,10/28/1962,300.0,46.72983°N 71.56304°E,46.729830,71.563040,150,https://en.wikipedia.org/wiki/Soviet_Project_K...
4,195 K5 (Joe 168),USSR,11/1/1962,300.0,46.3298°N 72.77929°E,46.329800,72.779290,59,https://en.wikipedia.org/wiki/Soviet_Project_K...
5,Starfish Prime,USA,7/9/1962,1400.0,16°28′N 169°38′W,16.466667,-169.633333,400,https://en.wikipedia.org/wiki/Starfish_Prime


In [11]:
# loop over each run
for i in range(len(df)):

    # load the run parameters
    run_id = df.iloc[i]['Name']
    date = datetime.strptime(df.iloc[i]['Date'], "%m/%d/%Y")
    yield_kt = df.iloc[i]['Yield (kt)']
    Latitude = df.iloc[i]['Latitude'] * np.pi / 180
    Longitude = df.iloc[i]['Longitude'] * np.pi / 180
    HOB = df.iloc[i]['Altitude (km)']

    # burst point
    Burst_Point = Point(
        EARTH_RADIUS + HOB,
        Latitude,
        Longitude,
        coordsys="lat/long geo",
    )

    # perform the scan
    result = region_scan(
        Burst_Point=Burst_Point,
        HOB=HOB,
        total_yield_kt=yield_kt,
        N_pts_phi=100,
        N_pts_lambd=100,
        N_pts_time=100,
        b_field_type='igrf',
    )

    with open(f"data/region_scan_{run_id}.pkl", "wb") as f:
        pickle.dump(result, f)

    # build the contour plot
    x, y, z = data_dic_to_xyz(result)
    contourf, levels = contour_plot(
        x, y, z, Burst_Point, save_path=f"figures/region_scan_{run_id}", grid=False, show=False,
    )

    # make a folium plot
    folium_plot(
        contourf,
        Burst_Point.phi_g * 180/np.pi,
        Burst_Point.lambd_g * 180/np.pi,
        levels,
        save_path=f"figures/region_scan_folium_{run_id}.png"
        )

  8%|▊         | 8/100 [1:11:03<13:35:27, 531.82s/it]